### Code to plot a telemetry profile (Beta!)
#### 2021 01 20 CJH

In [110]:
from networktables import NetworkTables
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

#### choose the address of the network tables server

In [61]:
# ip = '172.22.11.2' # robot
# ip = 'roboRIO-2429-FRC.local'
ip = 'localhost'  # simulation
NetworkTables.initialize(server=ip)

False

In [63]:
NetworkTables.addConnectionListener(connectionListener, immediateNotify=True)

ConnectionInfo(remote_id='Robot', remote_ip='localhost', remote_port=1735, last_update=358390.984, protocol_version=768) ; Connected=True


#### get the SmartDashboard table from the network tables server so we can query it

In [6]:
sd = NetworkTables.getTable("SmartDashboard")

#### get the odometry entries - currently set up as a list of 256 strings due to the limitations of what you can store in network tables 

In [31]:
example = sd.getString('Odometry',0)
print(f'Example: {example}\nx:{example[3:8]} y:{example[13:18]} rot:{example[25:31]} ')

Example: X: +2.15  Y: +5.91  Rot: +83.29
x:+2.15 y:+5.91 rot:+83.29 


In [ ]:
odometry_list = sd.getStringArray('Odometry_List',0)

### python is a high level language!
* list comprehension does for loops in one line
* pandas makes a great table (called dataframes) from a dictionary (or lists if you provide the column names)

In [90]:
x = [float(i[3:8]) for i in odometry_list]
y = [float(i[13:18]) for i in odometry_list]
rot = [float(i[25:31]) for i in odometry_list]
df = pd.DataFrame({'x':x, 'y':y, 'rotation':rot})

In [6]:
def get_data():
    # this was a lot simpler but stupid network tables can only take 256 values per array
        odometry_list = sd.getStringArray('Odometry_List',0)
        x = [float(i[3:8]) for i in odometry_list]
        y = [float(i[13:18]) for i in odometry_list]
        rot = [float(i[25:31]) for i in odometry_list]
        df = pd.DataFrame({'x':x, 'y':y, 'rotation':rot})
    return df

ConnectionInfo(remote_id='Robot', remote_ip='roboRIO-2429-FRC.local', remote_port=1735, last_update=0, protocol_version=768) ; Connected=True


#### take a quick look at the pandas datatable from the odometry

In [214]:
df[(df.index) % 20 == 0]

,x,y,rotation,delta
0,7.36,2.37,-103.20,NaN
20,10.80,0.30,-15.30,0.316228
40,13.30,2.60,153.00,0.223607
60,13.40,1.00,5.34,0.300000
80,14.20,5.40,92.30,0.300000
100,11.70,6.90,-177.00,0.316228
120,6.17,7.35,-154.80,0.282312
140,0.96,6.32,-126.10,0.212603
160,3.59,4.65,-142.60,0.205183
180,2.60,3.25,-15.76,0.291204


#### pandas is awesome

In [301]:
scale = 10 # doesn't do anything yet - quiver keeps autoscaling
df['delta'] = np.sqrt(np.asarray(df.diff()['x']**2) + np.asarray(df.diff()['y']**2))
df['radians'] = df['rotation'] * np.pi/180
df['vec_x'] = df['delta']* np.cos(df['radians'])*scale
df['vec_y'] = df['delta']* np.sin(df['radians'])*scale
df.at[0, 'vec_x'] = df.at[1, 'vec_x'] 
df.at[0, 'vec_y'] = df.at[1, 'vec_y']
df.at[0, 'delta'] = df.at[1, 'delta'] 
df[(df.index) % 20 == 0]

,x,y,rotation,delta,radians,vec_x,vec_y
0,7.36,2.37,-104.00,0.288617,-1.815142,-0.698229,-2.800442
20,10.80,0.30,-15.30,0.316228,-0.267035,3.050198,-0.834440
40,13.30,2.60,153.00,0.223607,2.670354,-1.992351,1.015154
60,13.40,1.00,5.34,0.300000,0.093201,2.986980,0.279197
80,14.20,5.40,92.30,0.300000,1.610939,-0.120395,2.997583
100,11.70,6.90,-177.00,0.316228,-3.089233,-3.157944,-0.165501
120,6.17,7.35,-154.80,0.282312,-2.701770,-2.554434,-1.202026
140,0.96,6.32,-126.10,0.212603,-2.200860,-1.252649,-1.717810
160,3.59,4.65,-142.60,0.205183,-2.488840,-1.630003,-1.246231
180,2.60,3.25,-15.76,0.291204,-0.275064,2.802574,-0.790936


In [340]:
def plot_df(df, arrows=True, save=False, fname='odometry.png'):
    label = "Mapping Odometry from Robot Simulation"
    fig, ax = plt.subplots(figsize=(16,10))
    df.plot.scatter(x='x', y='y',c='rotation', colormap='viridis', ax=ax, label='sim')
    if arrows:
        ax.quiver(x,y, df['vec_x'], df['vec_y'], df['rotation'])
    
    plt.text(df.iloc[[0]]['x'], df.iloc[[0]]['y']+.2, 'START', size=16)
    plt.text(df.iloc[[-1]]['x'], df.iloc[[-1]]['y']+.2, 'FINISH', size=16)
    
    ax.legend(loc='lower right', bbox_to_anchor=(0.1, 0.02), fontsize=14)
    ax.set_title(label, fontsize=16)
    ax.set_ylabel('y position on field', fontsize = 16)
    ax.set_xlabel('x position on field', fontsize = 16)
    plt.tight_layout()
    if save:
        plt.ioff()
        plt.savefig(fname, facecolor='w', bbox_inches='tight', dpi=100)
        plt.close()

    plt.show()


In [341]:
plot_df(df, arrows=True, save=True)

---
#### Pickle the data if desired (pickle writes python data to files for later use)

In [116]:
from datetime import datetime
time_stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
pickle_dir = r'.\pickle'
outfile = pickle_dir + '\\' + time_stamp +'.pkl'
with open(outfile, 'wb') as fp:
    pickle.dump(df, fp)

#### load data from pickle

In [118]:
# get a list of all the files 
import glob
pickle_dir = r'.\pickle'
files = glob.glob(pickle_dir+ '\*')
files

['.\\pickle\\20210120_144817.pkl', '.\\pickle\\20210120_144951.pkl']

In [224]:
infile = files[-1]
try:
    with open(infile, 'rb') as fp:
        df = pickle.load(fp)
except ValueError as e:
    print(f'Missing df in pickle {infile}, please create it and re-pickle')

In [225]:
df[(df.index) % 20 == 0]

,x,y,rotation
0,7.36,2.37,-103.20
20,10.80,0.30,-15.30
40,13.30,2.60,153.00
60,13.40,1.00,5.34
80,14.20,5.40,92.30
100,11.70,6.90,-177.00
120,6.17,7.35,-154.80
140,0.96,6.32,-126.10
160,3.59,4.65,-142.60
180,2.60,3.25,-15.76


In [ ]:
## other stuff to remember in case i need it
def valueChanged(table, key, value, isNew):
    print("valueChanged: key: '%s'; value: %s; isNew: %s" % (key, value, isNew))
def connectionListener(connected, info):
    print(info, "; Connected=%s" % connected)